In [20]:
import pandas as pd
import requests

In [48]:
with pd.HDFStore('./../data/bayarea_block_baseline.h5') as store:
    hh = store.households
    buildings = store.buildings

In [38]:
hh.columns

Index([u'taz', u'serialno', u'puma5', u'income', u'persons', u'hht',
       u'unittype', u'noc', u'bldgsz', u'tenure', u'vehicl', u'hinccat1',
       u'hinccat2', u'hhagecat', u'hsizecat', u'hfamily', u'hunittype',
       u'hnoccat', u'hwrkrcat', u'h0004', u'h0511', u'h1215', u'h1617',
       u'h1824', u'h2534', u'h3549', u'h5064', u'h6579', u'h80up', u'hworkers',
       u'hwork_f', u'hwork_p', u'huniv', u'hnwork', u'hretire', u'hpresch',
       u'hschpred', u'hschdriv', u'htypdwel', u'hownrent', u'hadnwst',
       u'hadwpst', u'hadkids', u'bucketbin', u'originalpuma', u'hmultiunit',
       u'building_id', u'block_id'],
      dtype='object')

In [47]:
hh.hownrent.value_counts()

1    1530771
2    1077248
Name: hownrent, dtype: int64

In [50]:
hh.groupby('hownrent').agg('mean')

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,hschpred,hschdriv,htypdwel,hadnwst,hadwpst,hadkids,bucketbin,originalpuma,hmultiunit,building_id
hownrent,,,,,,,,,,,,,,,,,,,,,
1,808.672174,4.915368e+06,2139.433298,99990.987053,2.768758,2.360737,0.0,0.490813,2.411738,1.248181,...,0.353837,0.078178,1.256245,0.146771,0.137275,0.151624,4.483121,2139.433298,0.120659,930178.741125
2,673.033416,4.930568e+06,2209.287363,54943.390530,2.497379,3.344824,0.0,0.492163,5.524158,3.029788,...,0.319015,0.063930,1.805625,0.167315,0.166755,0.127628,4.504253,2209.287363,0.732123,915855.801941


In [49]:
buildings

,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,nonres_rent_per_sqft,res_price_per_sqft,stories,year_built,redfin_sale_price,redfin_sale_year,redfin_home_type,costar_property_type,costar_rent,building_type_id,block_id
building_id,,,,,,,,,,,,,,,,,,,
1,742974,1,0.00,1,2029,2029.42425,0,2029.42425,0.000000,302.769751,1,1945,NaN,NaN,,,,1,060411141003012
2,744961,1,0.00,1,2029,2029.42425,0,2029.42425,0.000000,254.429279,1,1965,NaN,NaN,,,,1,060411311003059
3,1442641,1,53262.87,1,1568,1568.00000,0,1568.00000,0.000000,183.474166,1,1964,NaN,NaN,,,,1,060855082023014
4,190969,2,245000.00,0,0,1266.00000,1595,1266.00000,0.000000,0.000000,2,1992,340000.0,2003.0,Condo/Coop,,,3,060014308004003
5,308709,2,283500.00,0,0,1513.00000,1513,1513.00000,0.000000,0.000000,1,1978,442000.0,2004.0,Condo/Coop,,,3,060014418002005
6,308688,2,304500.00,0,0,1717.00000,1717,1717.00000,0.000000,0.000000,1,1978,620000.0,2006.0,Condo/Coop,,,3,060014418002006
7,328712,1,0.00,0,0,1393.00000,0,0.00000,0.000000,0.000000,1,2008,670250.0,2008.0,Single Family Residential,,,1,060014415031233
8,369900,2,0.00,0,0,1387.00000,0,0.00000,0.000000,0.000000,1,1996,426000.0,2005.0,Townhouse,,,3,060014505022004
9,742661,1,116580.00,1,1018,1018.00000,0,1018.00000,0.000000,474.350534,1,1946,703000.0,2007.0,Single Family Residential,,,1,060411141003015


In [41]:
hh.income.describe()

count    2.608019e+06
mean     8.138398e+04
std      8.149481e+04
min     -2.000000e+04
25%      3.020000e+04
50%      6.090000e+04
75%      1.020000e+05
max      1.968504e+06
Name: income, dtype: float64

In [28]:
# Job Density

options = {'area_unit':'miles'}

data = pd.Series(requests.post('http://localhost:5000/model_data/base/density/jobs/tracts', options).json())
df = pd.DataFrame(data, columns=['job_density'])

In [30]:
# Population Density

options = {'area_unit':'miles'}

data = pd.Series(requests.post('http://localhost:5000/model_data/base/density/population/tracts', options).json())
df['pop_density'] = data

In [42]:
# Median Income

options = {'target_var':'income',
          'spatial_agg':'median'}

data = pd.Series(requests.post('http://localhost:5000/model_data/base/descriptive/households/tracts', options).json())
df['income_median'] = data

In [44]:
# Mean Income

options = {'target_var':'income',
          'spatial_agg':'mean'}

data = pd.Series(requests.post('http://localhost:5000/model_data/base/descriptive/households/tracts', options).json())
df['income_mean'] = data

In [51]:
# Number of Households

data = pd.Series(requests.get('http://localhost:5000/model_data/base/descriptive/households/tracts').json())
df['households'] = data

In [58]:
# Number of Persons

options = {'target_var':'persons',
          'spatial_agg':'sum'}

data = pd.Series(requests.post('http://localhost:5000/model_data/base/descriptive/households/tracts', options).json())
df['persons'] = data

In [60]:
# Number of Jobs

data = pd.Series(requests.post('http://localhost:5000/model_data/base/descriptive/jobs/tracts').json())
df['jobs'] = data

In [62]:
# Households Renting
options = {'hownrent':'2'}
data = pd.Series(requests.post('http://localhost:5000/model_data/base/descriptive/households/tracts', options).json())
df['hh_rent'] = data

In [64]:
# Households Own
options = {'hownrent':'1'}
data = pd.Series(requests.post('http://localhost:5000/model_data/base/descriptive/households/tracts', options).json())
df['hh_own'] = data

In [65]:
df

,job_density,pop_density,income_median,income_mean,households,persons,jobs,hh_rent,hh_own
0600140010,689.058298,1108.665072,114025.0,137803.985141,1346.0,2946.0,1831.0,212.0,1134.0
0600140020,22809.595446,8168.977419,74100.0,96481.841981,848.0,1877.0,5241.0,334.0,514.0
0600140030,3854.539340,11343.358630,54000.0,78521.399752,2419.0,4841.0,1645.0,1197.0,1222.0
0600140040,4228.999517,14851.186292,54210.0,68762.547696,1866.0,4035.0,1149.0,1068.0,798.0
0600140050,2946.215888,15757.190503,37730.0,56054.495226,1571.0,3578.0,669.0,976.0,595.0
0600140060,1347.797217,14260.564100,40000.0,56642.791304,690.0,1640.0,155.0,435.0,255.0
0600140070,2029.602879,12071.724951,30000.0,42296.902884,1699.0,4104.0,690.0,1106.0,593.0
0600140080,2794.026437,15847.717952,34710.0,50297.825263,1900.0,4254.0,750.0,1247.0,653.0
0600140090,1279.268139,14254.702126,28000.0,41075.709384,991.0,2340.0,210.0,629.0,362.0
0600140100,6072.582842,13222.342242,30000.0,38307.525235,2338.0,5892.0,2706.0,1584.0,754.0
